In [1]:
from bertopic import BERTopic
import pandas as pd
from tqdm import tqdm

path = 'path-to-data-repository'

In [2]:
topic_model = BERTopic.load("{}topic_model03-05".format(path))

In [3]:
df = pd.read_parquet('{}pdf-phrases03-05.parquet'.format(path))
df

,doi,file_name,text,phrases
0,10.1016/j.biortech.2010.01.041,10.1016@j.biortech.2010.01.041.pdf,Geranylation of benzoic acid derivatives by en...,[Geranylation of benzoic acid derivatives by e...
1,10.1590/S0100-40422010000200017,10.1590@S0100-40422010000200017.pdf,"Quim. Nova, Vol. 33, No. 2, 321-323, 2010 Arti...","[Quim. Nova, Vol. 33, No. 2, 321-323, 2010 Art..."
2,10.1590/S0103-50532006000500017,10.1590@S0103-50532006000500017.pdf,"J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934, ...","[J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934,..."
3,10.1002/pca.818,10.1002@pca.818.pdf,FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM CR...,[FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM C...
4,10.1002/ps.478,10.1002@ps.478.pdf,Biological activity of astilbin from ���������...,[Biological activity of astilbin from ��������...
...,...,...,...,...
385,10.1016/S0031-9422(00)94671-5,10.1016@S0031-9422(00)94671-5.pdf,"Short Reports 1671 Dichrographe Mark III, R....","[Short Reports 1671 Dichrographe Mark III, R..."
386,10.1016/S0031-9422(00)80444-6,10.1016@S0031-9422(00)80444-6.pdf,"Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296,...","[Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296..."
387,10.1021/np50032a021,10.1021@np50032a021.pdf,Journal of Natural Products Vol. 47. No. 2. p...,[Journal of Natural Products Vol. 47. No. 2. ...
388,10.1016/S0031-9422(00)83467-6,10.1016@S0031-9422(00)83467-6.pdf,"Phytockmistry, Vol. 23, No. 8, pp. 1677-1679,...","[Phytockmistry, Vol. 23, No. 8, pp. 1677-1679..."


In [4]:
topics = []
for phrases_list in tqdm(df.phrases.values):
    phrases_topics = []
    for phrase in phrases_list:
        phrases_topics.append(topic_model.find_topics(phrase)[0])
    topics.append(list(set([item for sublist in phrases_topics for item in sublist])))

100%|██████████| 390/390 [25:23<00:00,  3.91s/it]


In [22]:
from copy import deepcopy

topics_flat = pd.Series([item for sublist in topics for item in sublist])
topics_count = {}
for topic in topics_flat.unique():
    topics_count[topic] = len(topics_flat[topics_flat == topic])/len(topics_flat.unique())
to_remove = [-1, 0]
for key, value in topics_count.items():
    if key == -1 or key == 0:
        continue
    if value > 0.8:
        to_remove.append(key)
print("to remove: {}".format(len(to_remove)))
topics_filtered = deepcopy(topics)
for index, topic_list in enumerate(topics_filtered):
    new_list = []
    for topic in topic_list:
        if topic not in to_remove:
            new_list.append(topic)
    topics_filtered[index] = new_list
print("topics flat original: {}".format(len(topics_flat)))
print("topics flat filtered: {}".format(len([item for sublist in topics_filtered for item in sublist])))


to remove: 59
topics flat original: 17053
topics flat filtered: 2676


In [23]:
df['topics'] = topics_filtered
df

,doi,file_name,text,phrases,topics
0,10.1016/j.biortech.2010.01.041,10.1016@j.biortech.2010.01.041.pdf,Geranylation of benzoic acid derivatives by en...,[Geranylation of benzoic acid derivatives by e...,"[128, 131, 137, 9, 25, 42, 50, 51, 52, 70, 106..."
1,10.1590/S0100-40422010000200017,10.1590@S0100-40422010000200017.pdf,"Quim. Nova, Vol. 33, No. 2, 321-323, 2010 Arti...","[Quim. Nova, Vol. 33, No. 2, 321-323, 2010 Art...","[140, 50, 89, 106]"
2,10.1590/S0103-50532006000500017,10.1590@S0103-50532006000500017.pdf,"J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934, ...","[J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934,...","[137, 140, 27, 51, 63, 75, 115]"
3,10.1002/pca.818,10.1002@pca.818.pdf,FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM CR...,[FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM C...,"[137, 140, 16, 50, 51, 66, 90, 100]"
4,10.1002/ps.478,10.1002@ps.478.pdf,Biological activity of astilbin from ���������...,[Biological activity of astilbin from ��������...,"[1, 7, 140, 17, 23, 25, 29, 31, 36, 37, 50, 57..."
...,...,...,...,...,...
385,10.1016/S0031-9422(00)94671-5,10.1016@S0031-9422(00)94671-5.pdf,"Short Reports 1671 Dichrographe Mark III, R....","[Short Reports 1671 Dichrographe Mark III, R...","[15, 64, 70, 124, 126]"
386,10.1016/S0031-9422(00)80444-6,10.1016@S0031-9422(00)80444-6.pdf,"Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296,...","[Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296...","[35, 51, 63]"
387,10.1021/np50032a021,10.1021@np50032a021.pdf,Journal of Natural Products Vol. 47. No. 2. p...,[Journal of Natural Products Vol. 47. No. 2. ...,[90]
388,10.1016/S0031-9422(00)83467-6,10.1016@S0031-9422(00)83467-6.pdf,"Phytockmistry, Vol. 23, No. 8, pp. 1677-1679,...","[Phytockmistry, Vol. 23, No. 8, pp. 1677-1679...","[137, 31, 51, 82, 89, 90]"


In [24]:
import numpy as np
len_list = [len(topic_list) for topic_list in df.topics.values]
df.iloc[len_list.index(np.max(len_list))]

doi                         10.1023/B:JOEC.0000042400.14451.08
file_name               10.1023@B:JOEC.0000042400.14451.08.pdf
text         Journal of Chemical Ecology, Vol. 30, No. 9, S...
phrases      [Journal of Chemical Ecology, Vol. 30, No. 9, ...
topics       [128, 3, 9, 140, 16, 22, 24, 25, 27, 34, 35, 4...
Name: 224, dtype: object

In [25]:
df.to_parquet('{}topics03-05.parquet'.format(path))